In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
%matplotlib inline
import os, time

pd.set_option('display.max_columns', None)
import plotly.express as px

In [2]:
from IPython.display import HTML

In [3]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [4]:
df_shelv = pd.read_excel('shelving.xlsx')
df_shelv.rename(columns={'Location':'DSP_LOCN','Unnamed: 1':'Shelving'},inplace=True)
df_shelv.head()

,DSP_LOCN,Shelving
0,4S00010101,SHELVING
1,4S00010102,SHELVING
2,4S00010103,SHELVING
3,4S00010104,SHELVING
4,4S00010105,SHELVING


In [5]:
df_S_NS = pd.read_csv('exportFG_FOC.csv')
df_S_NS.head()

,ITEM_NAME,SALE_GRP
0,S1442800,S
1,S1442801,S
2,S1532600,S
3,S1663800,S
4,S1663801,S


In [6]:
df_FPE = pd.read_excel('FPE.xlsx')
df_FPE.rename(columns={'DSP_LOCN':'Loc'},inplace=True)
df_FPE.rename(columns={'Location':'DSP_LOCN'},inplace=True)
df_FPE.head()

,DSP_LOCN,Level,Aisle,Bay,Side,Level.1,Position,HP/FP,FPE,Velocity,Loc,Arrow Direction,PICKING SEQ RACKING,PutAway Sequence RACKING,LOCN_CLASS,Active/Reserve,LENGTH,WIDTH,HEIGTH,Code(Reserve Loc),Putaway Zone(Reserve Loc),countif,Unnamed: 22
0,4P02011001,4,2,1,Odd,10,1,FP,1.0,B,4-P-02-01-10-01,UP,PK00001,NaN,R,Reserve,120,100,120,OVR,SG9LTOVR-OVERFLOW,1,NaN
1,4P02011002,4,2,1,Odd,10,2,FP,1.0,B,4-P-02-01-10-02,UP,PK00002,NaN,R,Reserve,120,100,120,OVR,SG9LTOVR-OVERFLOW,1,NaN
2,4P02011003,4,2,1,Odd,10,3,FP,1.0,B,4-P-02-01-10-03,UP,PK00003,NaN,R,Reserve,120,100,120,OVR,SG9LTOVR-OVERFLOW,1,NaN
3,4P02011203,4,2,1,Odd,12,3,FP,1.0,B,4-P-02-01-12-03,UP,PK00004,NaN,R,Reserve,120,100,120,OVR,SG9LTOVR-OVERFLOW,1,NaN
4,4P02011202,4,2,1,Odd,12,2,FP,1.0,B,4-P-02-01-12-02,UP,PK00005,NaN,R,Reserve,120,100,120,OVR,SG9LTOVR-OVERFLOW,1,NaN


In [ ]:
path = "C:\\Users\\btg168\\Desktop\\Inventory reports test\\"
d_init={}

for file in os.listdir(path):
    d_init[file] = pd.read_excel(path+file)
    d_init[file]['Date File created'] = time.ctime(os.path.getmtime(path+file))
    d_init[file]['Date File created'] = pd.to_datetime(d_init[file]['Date File created'])
    d_init[file]['Date File created'] = d_init[file]['Date File created'].dt.date


In [ ]:
df_concat_raw = pd.concat(d_init.values())

In [ ]:
df_concat.dtypes

In [ ]:
df_concat_raw.shape

In [ ]:
df_concat['DSP_LOCN'].unique()

In [ ]:
df_concat_filtered = df_concat_raw.merge(df_shelv,on='DSP_LOCN',how='left')
df_concat_filtered = df_concat_filtered.merge(df_S_NS,on='ITEM_NAME',how='left' )
df_concat_filtered = df_concat_filtered.merge(df_FPE[['DSP_LOCN','HP/FP']],on='DSP_LOCN',how='left' )
df_concat_filtered.dropna(subset=['DSP_LOCN'],inplace=True)
df_concat_filtered[['REF_FIELD1']] = df_concat_filtered['REF_FIELD1'].replace(to_replace=\
                                                                          ['HR','R. LAUREN','URBAN DECAY','KERASTASE',\
                                                                           'SKINCEUTICALS','Martin MARGIELA','VICHY',\
                                                                           'Atelier Cologne','VIKTOR ET ROLF',\
                                                                           'ROCHE POSAY','HOUSE 99',\
                                                                           'IMARQUES INTER-DEPARTMENT','CLARISONIC']\
                                                                          ,value = 'Mixed Brands')
df_concat_filtered['to keep'] = df_concat_filtered['DSP_LOCN'].apply(lambda x: False if len(x)-sum(c.isdigit() for c in x)>2 else True)
df_concat_filtered = df_concat_filtered.loc[(df_concat_filtered.DSP_LOCN.str[:2] != 'LT')\
                                        & (df_concat_filtered['INVENTORY_TYPE']=='U') & (df_concat_filtered['ZINDEX']==1)\
                                        & (df_concat_filtered['Shelving'].isna()) & (df_concat_filtered['to keep']== True)\
                                        & (df_concat_filtered.DSP_LOCN.str[:1] == '4')\
                                        & (df_concat_filtered['REF_FIELD1'] != "L'OREAL PARIS")]
df_concat_filtered = df_concat_filtered[['REF_FIELD1','DSP_LOCN','ITEM_NAME','SALE_GRP','Date File created','HP/FP']]

# Evolution of brands' ratio occupancy over the time

In [ ]:
fig,axes = plt.subplots(len(df_concat_filtered['REF_FIELD1'].unique()),figsize=(12,30))
fig.tight_layout(pad=7.0)
i = 0

for ref in list(df_concat_filtered['REF_FIELD1'].unique()):
    df_plot = df_concat_filtered[df_concat_filtered['REF_FIELD1'] ==f'{ref}'].groupby(['REF_FIELD1','Date File created'])[['DSP_LOCN']].nunique().reset_index()
    df_plot = df_plot[['REF_FIELD1','DSP_LOCN','Date File created']]
    df_plot.plot(kind='bar',ax=axes[i],x='Date File created')
    axes[i].set_xlabel('')
    axes[i].axhline(y=df_plot['DSP_LOCN'].mean(), color='g',label = f"average number of locations: {round(df_plot['DSP_LOCN'].mean())}")
    axes[i].legend(loc ='lower left')
    axes[i].set_title(f"{ref}")
    
    i+=1
    

In [ ]:
data = {'Mixed Brands':[770*2],'BIOTHERM':[798+693]\
        ,'SHU UEMURA':[728+462],'YSL':[462+847+847],'G. ARMANI':[882+770],\
        'KIEHLS':[770+840+770+770+798],'LANCOME':[693+728+756+735+693+798+684],"L'OREAL PARIS":[846*2+539*2+756+692+315]}
df_master = pd.DataFrame(data,columns=data.keys()).T
df_master.reset_index(inplace=True)
df_master.rename(columns={'index':'REF_FIELD1'},inplace=True)
df_master.rename(columns={df_master.columns[-1]:'Max Occupancy'},inplace=True)
#master from Vincent LAU
# print('\nMaster location from Vincent LAU')
print('\nMaster location from Vincent SIA \nLocation wise (no FPE)')
df_master

In [ ]:
fig,axes = plt.subplots(len(df_concat['REF_FIELD1'].unique()),figsize=(10,30))
fig.tight_layout(pad=7.0)
i = 0

for ref in list(df_concat['REF_FIELD1'].unique()):
    df_plot = df_concat[df_concat['REF_FIELD1'] =='{}'.format(ref)].groupby(['REF_FIELD1','Date File created'])[['DSP_LOCN']].nunique().reset_index()
    df_plot = df_plot.merge(df_master, on='REF_FIELD1', how='left')
    df_plot['Ratio'] = df_plot.apply(lambda x: x['DSP_LOCN']*100/x['Max Occupancy'], axis=1)
    df_plot = df_plot[['REF_FIELD1','Ratio','Date File created']]
    df_plot.plot(kind='bar',ax=axes[i],x='Date File created')
    axes[i].set_xlabel('')
    axes[i].axhline(y=100, color='r',label='100% occupancy')
    axes[i].axhline(y=df_plot['Ratio'].mean(), color='g',label = f"average occupancy: {round(df_plot['Ratio'].mean())} %")
    axes[i].legend(loc ='lower left')
    axes[i].set_title(f"{ref}")
    
    i+=1
    

# Average ratio of occupancy of each brand

In [ ]:
df = df_concat.copy()

In [ ]:
df = df.groupby(['Date File created','REF_FIELD1'])[['DSP_LOCN']].nunique()
df.rename(columns={'DSP_LOCN':'Nbr of loc'},inplace=True)
df.reset_index(inplace=True)

In [ ]:
df = df.groupby('REF_FIELD1')[['Nbr of loc']].mean()
df = df.merge(df_master,on='REF_FIELD1', how='left')
df['Average Ratio'] = round(df['Nbr of loc']*100/df['Max Occupancy'],1)
df = df.sort_values(by='Average Ratio',ascending=False)
df['Max Occupancy'] = pd.to_numeric(df['Max Occupancy'])
df

In [ ]:
df['85% occupancy'] = df['Max Occupancy']*0.85
df['Nbr of loc to add'] = df[['Nbr of loc','85% occupancy']].apply(lambda x: x['Nbr of loc'] - x['85% occupancy'] if x['Nbr of loc'] - x['85% occupancy']>0 else 0, axis=1)
df

In [ ]:
df['Average Ratio'].mean()

In [ ]:
df['Nbr of loc to remove'].sum()